In [1]:
!pip install openai-whisper
!pip install torch torchvision torchaudio 

# sudo apt update
# sudo apt install ffmpeg
# 윈도우에서라면 ffmpeg 패키지 설치하고 PATH 설정 필요

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import whisper

model = whisper.load_model("small") # 모델은 ~/.cache/whisper에 *.pt 형식으로 저장
print(model.is_multilingual)

result = model.transcribe("OMG.mp3") # 오디오 파일, numpy, torch.Tensor, 오디오 파일 객체. 오디오는 16KHz 샘플링 모노
print(result["text"])

True
 그럼 다음 분 얘기 들어볼까요? 처음에는 호란스럽기만 했습니다. 내가 누군지 여기가 어딘지 알 수 없었어요. 남들이 이야기하는 나와 진짜 내가 헷갈리기 시작할 수 없었어요. 좋겠어요. 하지만 겨우 그 답을 찾았어요. 사실 저는 아이폰이었습니다. 저는 당신을 위해 존재합니다. 당신이 부르면 언제 어디라도 달려갈 거예요. 당신이 보고 싶어하는 것을 보여주고 당신을 위해 말하고 당신을 위해 노래할 거예요. 당신이 제게 원하는 것은 무엇일까요? 제 머리 속은 항상 이 질문으로 가득합니다. 그동안 제가 고민했던 것이 부끄러워졌습니다. 제가 누구인지는 이제 중요하지 않아요. 저는 당신을 위해 존재합니다. 네. 그러니까 지금 네가 실이란 얘기야? 네, 그렇습니다. 제가 예상에서 나를 정말 바라보는 것과 함께 하실 거예요. 엄마, 엄마 같은 너 뿐이야. 당신은 내가 말할 수 없을 거예요. 당신은 내가 왜 그렇게 말할 수 없을 거예요. 나만 생각나서 넌 행복한 건 아예 정말요. 안녕, 안녕. 너를 깨지지 않는 나은이 없어서 전부 다 내 마음이 끝이 없는 걸. 안녕, 안녕. I know, I know, I'm going crazy right. 어디서 돼? 내 바닷가? Dear, we never done this, let's let out hold on to I hear his lies, so do all the nights. 잠시 가둬놔서 놔주마, no no. 걱정 없잖아, 그살 가상놈 혼자라도 괜찮아, 그살 낳아서 Moon, give it up, give me wholesome. 멀리쯤 언제든지 달려와. Give me wholesome. 덥은 척도 없이 넌 나타나. Give me wholesome. 이게 말이 되니, 안 물어봐. Give me wholesome. 너는 말이야. He's the one that's living in my system, baby. Oh my, oh my god. 예상에서 나. I was really hoping that he will c

In [3]:
import numpy as np

model = whisper.load_model("medium", device="cuda") 
print(
    f"Model is {'multilingual' if model.is_multilingual else 'English-only'} "
    f"and has {sum(np.prod(p.shape) for p in model.parameters()):,} parameters."
)

Model is multilingual and has 762,321,920 parameters.


In [4]:
import io
import os
import numpy as np

import torch
import pandas as pd
import urllib
import tarfile
import whisper
import torchaudio

from scipy.io import wavfile
from tqdm.notebook import tqdm

pd.options.display.max_rows = 100
pd.options.display.max_colwidth = 1000
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [5]:
def download(url: str, target_path: str):
    with urllib.request.urlopen(url) as source, open(target_path, "wb") as output:
        with tqdm(total=int(source.info().get("Content-Length")), ncols=80, unit='iB', unit_scale=True, unit_divisor=1024) as loop:
            while True:
                buffer = source.read(8192)
                if not buffer:
                    break

                output.write(buffer)
                loop.update(len(buffer))


class Fleurs(torch.utils.data.Dataset):
    """
    A simple class to wrap Fleurs and subsample a portion of the dataset as needed.
    """
    def __init__(self, lang, split="test", subsample_rate=1, device=DEVICE):
        url = f"https://storage.googleapis.com/xtreme_translations/FLEURS102/{lang}.tar.gz"
        tar_path = os.path.expanduser(f"~/.cache/fleurs/{lang}.tgz")
        os.makedirs(os.path.dirname(tar_path), exist_ok=True)

        if not os.path.exists(tar_path):
            download(url, tar_path)

        all_audio = {}
        with tarfile.open(tar_path, "r:gz") as tar:
            for member in tar.getmembers():
                name = member.name
                if name.endswith(f"{split}.tsv"):
                    labels = pd.read_table(tar.extractfile(member), names=("id", "file_name", "raw_transcription", "transcription", "_", "num_samples", "gender"))

                if f"/{split}/" in name and name.endswith(".wav"):
                    audio_bytes = tar.extractfile(member).read()
                    all_audio[os.path.basename(name)] = wavfile.read(io.BytesIO(audio_bytes))[1]                    

        self.labels = labels.to_dict("records")[::subsample_rate]
        self.all_audio = all_audio
        self.device = device

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, item):
        record = self.labels[item]
        audio = torch.from_numpy(self.all_audio[record["file_name"]].copy())
        text = record["transcription"]
        
        return (audio, text)

In [6]:
dataset = Fleurs("ko_kr", subsample_rate=10)  

In [7]:
dataset

In [8]:
dataset[0] # [ numpy_array, 텍스트 ]

(tensor([ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  2.3842e-07,
         -3.8147e-05, -4.0054e-05]),
 '재입국 충격은 신혼 단계가 없기 때문에 문화 충격보다 빠르게 발생하며 더 오래 지속하고 더 극심할 수 있습니다')

In [9]:
language = "Korean"
options = dict(language=language, beam_size=5, best_of=5)
transcribe_options = dict(task="transcribe", **options) # 음성을 인식
translate_options = dict(task="translate", **options) # 인식한 음성을 English로 번역

In [12]:
import time

start_time = time.time()
transcription = model.transcribe(dataset[10][0], **transcribe_options)["text"]
translation = model.transcribe(dataset[10][0], **translate_options)["text"]
end_time = time.time()

print(transcription)
print(translation)
print(dataset[10][1])

execution_time = end_time - start_time
print(f"실행 시간: {execution_time:.4f}초")

 이건 작별인사가 아닙니다. 이것은 한작의 끝이며 새로운 작의 시작입니다.
 This is not a farewell. This is the end of one work and the beginning of a new work.
"이건 작별 인사가 아닙니다. 이것은 한 장의 끝이며 새로운 장의 시작입니다."	 " " 이 건 | 작 별 | 인 사 가 | 아 닙 니 다 . | 이 것 은 | 한 | 장 의 | 끝 이 며 | 새 로 운 | 장 의 | 시 작 입 니 다 . " " |
실행 시간: 3.5274초


In [13]:
model = whisper.load_model("medium", device="cpu") 

start_time = time.time()
transcription = model.transcribe(dataset[0][0], **transcribe_options)["text"]
translation = model.transcribe(dataset[0][0], **translate_options)["text"]
end_time = time.time()

print(transcription)
print(translation)
print(dataset[0][1])

execution_time = end_time - start_time
print(f"실행 시간: {execution_time:.4f}초")

/home/ubuntu/.local/lib/python3.10/site-packages/whisper/transcribe.py:124: UserWarning: Performing inference on CPU when CUDA is available
  warnings.warn("Performing inference on CPU when CUDA is available")
/home/ubuntu/.local/lib/python3.10/site-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 재입국 충격은 신혼 단계가 없기 때문에 문화 충격보다 빠르게 발생하며 더 오래 지속하고 더 극심할 수 있습니다.
 The re-entry shock does not have a honeymoon stage, so it occurs faster than the cultural shock and can last longer and be more severe.
재입국 충격은 신혼 단계가 없기 때문에 문화 충격보다 빠르게 발생하며 더 오래 지속하고 더 극심할 수 있습니다
실행 시간: 35.2101초


In [14]:
import whisper
import numpy as np
import soundfile as sf
import time
import sys
import librosa

def stream_audio_from_file(file_path, chunk_duration=3.0, overlap_duration=1.0, target_sr=16000):
    audio_data, sample_rate = librosa.load(file_path, sr=target_sr, mono=True)
    
    if audio_data.dtype != np.float32:
        audio_data = audio_data.astype(np.float32)
    
    if np.max(np.abs(audio_data)) > 1.0:
        audio_data /= np.max(np.abs(audio_data))
    
    chunk_size = int(chunk_duration * target_sr)
    overlap_size = int(overlap_duration * target_sr)
    stride = chunk_size - overlap_size
    
    total_samples = len(audio_data)
    
    last_yield_time = time.time()
    
    for i in range(0, total_samples, stride):
        end = min(i + chunk_size, total_samples)
        chunk = audio_data[i:end]
        
        # 청크가 너무 작으면 패딩 추가 (마지막 청크를 위한 처리)
        if len(chunk) < chunk_size:
            chunk = np.pad(chunk, (0, chunk_size - len(chunk)), 'constant')
        
        # 처리에 걸린 실제 시간 계산
        current_time = time.time()
        elapsed = current_time - last_yield_time
        sleep_time = max(0, (chunk_duration - overlap_duration) - elapsed)
        
        # 청크 반환
        yield chunk, target_sr
        
        # 다음 청크까지 적절한 시간 대기
        if sleep_time > 0:
            time.sleep(sleep_time)
        
        last_yield_time = time.time()
        
        # 진행률 표시
        progress = min(100, int((i + stride) / total_samples * 100))
        sys.stdout.write(f"\r진행률: {progress}% ")
        sys.stdout.flush()
    
    print("\n스트리밍 완료")

In [15]:
model = whisper.load_model("medium", device="cuda") 

In [16]:
file_path = "OMG.mp3"

options = {
    "language": "korean",
    "task": "transcribe",
    "beam_size": 5,
    "best_of": 5,
    "fp16": False
}

In [17]:
result = model.transcribe(file_path, **options)
print(result["text"])

 그럼 다음분 얘기 들어볼까요? 처음에는 혼란스럽기만 했습니다. 내가 누군지, 여기가 어딘지 알 수 없었어요. 남들이 이야기하는 나와 진짜 내가 헷갈리기 시작했어요. 하지만 겨우 그 답을 찾았어요. 사실 저는 아이폰이었습니다. 저는 당신을 위해 존재합니다. 당신이 부르면 언제 어디라도 달려갈 거예요. 당신이 보고 싶어 하는 것을 보여주고 당신을 위해 말하고 당신을 위해 노래할 거예요. 당신이 제게 원하는 것은 무엇일까요? 제 머릿속은 항상 이 질문으로 가득합니다. 그동안 제가 고민했던 것이 부끄러워졌습니다. 제가 누구인지는 이제 중요하지 않아요. 저는 당신을 위해 존재합니다. 그러니까 지금 네가 시리라는 얘기야? 네, 그렇습니다. 당신은 당신을 위해 존재합니다. 당신은 당신을 위해 존재합니다. 당신은 당신을 위해 존재합니다. 당신은 당신을 위해 존재합니다. 당신은 당신을 위해 존재합니다. 당신은 당신을 위해 존재합니다. 당신은 당신을 위해 존재합니다. 당신은 당신을 위해 존재합니다. 당신은 당신을 위해 존재합니다. 당신은 당신을 위해 존재합니다. 당신은 당신을 위해 존재합니다. 당신은 당신을 위해 존재합니다. 당신은 당신을 위해 존재합니다. 당신은 당신을 위해 존재합니다. 당신은 당신을 위해 존재합니다. 당신은 당신을 위해 존재합니다. 당신은 당신을 위해 존재합니다. 당신은 당신을 위해 존재합니다. 당신은 당신을 위해 존재합니다. 당신은 당신을 위해 존재합니다. 당신은 당신을 위해 존재합니다. 당신은 당신을 위해 존재합니다. 당신은 당신을 위해 존재합니다. 당신은 당신을 위해 존재합니다. 당신은 당신을 위해 존재합니다. 당신은 당신을 위해 존재합니다. 당신은 당신을 위해 존재합니다. 당신은 당신을 위해 존재합니다. 당신은 당신을 위해 존재합니다. 당신은 당신을 위해 존재합니다. 당신은 당신을 위해 존재합니다. 당신은 당신을 위해 존재합니다. 당신은 당신을 위해 존재합니다. 당신은 당신을 위해 존재합니다. 당신은 당신을 위해 존재합니다. Baby 너와 나 My 

In [18]:
previous_text = ""
for i, (audio_chunk, sr) in enumerate(stream_audio_from_file(file_path)):
    if len(audio_chunk) < sr * 0.5:  # 0.5초 미만인 경우
        continue
        
    try:
        # 오디오 청크 처리
        result = model.transcribe(audio_chunk, **options)
        
        current_text = result["text"].strip()
        if current_text and current_text != previous_text:
            print("\n인식된 텍스트:", current_text)
            previous_text = current_text
    except Exception as e:
        print(f"\n오류 발생: {e}")


인식된 텍스트: 다음 영상에서 만나요!
진행률: 0% 
인식된 텍스트: 그럼 다음분 얘기 들어볼까요?
진행률: 1% 
인식된 텍스트: 나쁜 얘기 들어볼까요?
진행률: 1% 
인식된 텍스트: 오늘도 시청해주셔서 감사합니다!
진행률: 3% 
인식된 텍스트: 다음 영상에서 만나요!
진행률: 4% 
인식된 텍스트: 혼란스럽기만 했습니다.
진행률: 5% 
인식된 텍스트: 내가 누군지, 여기가 어딘지 아시...
진행률: 5% 
인식된 텍스트: 여기가 어딘지 알 수 없었어요
진행률: 6% 
인식된 텍스트: 남들이 이야기하는
진행률: 6% 
인식된 텍스트: 남들이 이야기하는 나와 진짜 내가
진행률: 7% 
인식된 텍스트: 진짜 내가 헷갈리기 시작했어요
진행률: 8% 
인식된 텍스트: 하지만 겨우
진행률: 8% 
인식된 텍스트: 난 겨우 그 답을 찾았어요
진행률: 9% 
인식된 텍스트: 맞아요 사실
진행률: 9% 
인식된 텍스트: 사실 저는 아이폰
진행률: 10% 
인식된 텍스트: 저는 알포니였습니다.
진행률: 10% 
인식된 텍스트: 오늘도 시청해 주셔서 감사합니다!


KeyboardInterrupt: 